In [28]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/joaquin/Documents/GitHub/skforecast


In [29]:
from skforecast.drift_detection import RangeDriftDetector
from pprint import pprint
from copy import deepcopy

In [30]:
y = pd.Series(np.random.rand(5000), index=pd.date_range(start='2000-01-01', periods=5000, freq='D'), name="y")
series_df = pd.DataFrame(
    {
        'series_1': pd.Series(np.random.rand(5000), index=pd.date_range(start='2000-01-01', periods=5000, freq='D')),
        'series_2': pd.Series(np.random.rand(5000), index=pd.date_range(start='2000-01-01', periods=5000, freq='D')),
    }
)
series_df_long = series_df.stack().to_frame().rename(columns={0: 'y'}).swaplevel(0,1).sort_index()
exog = pd.Series(np.random.rand(5000), index=y.index, name="temp")
exog_df = pd.DataFrame(index=y.index)
for i in range(5):
    exog_df[f"feature_{i}"] = np.random.rand(5000)
for i in range(5):
    exog_df[f"cat_feature_{i}"] = np.random.choice(["A", "B", "C"], 5000)


series_dict = series_df.to_dict(orient='series')
exog_dict = {'series_1': exog_df, 'series_2': exog_df}
exog_df_long = [
    exog_df.copy().assign(series_id='series_1'),
    exog_df.copy().assign(series_id='series_2')
]
exog_df_long = pd.concat(exog_df_long).set_index('series_id', append=True).reorder_levels(['series_id', None]).sort_index()

In [31]:
np.unique(y)

array([6.68106393e-05, 1.52118493e-04, 4.53889134e-04, ...,
       9.99581581e-01, 9.99845813e-01, 9.99984925e-01], shape=(5000,))

In [33]:
X = pd.Series([0, np.nan, 15])
result = RangeDriftDetector._check_feature_range(feature_range, X)
assert result == True

In [45]:
# Categorical with NaN - NaN is treated as a unique value
feature_range = {'a', 'b', 'c'}
X = pd.Series(['a', np.nan, 'b'])
result = RangeDriftDetector._check_feature_range(feature_range, X)
set(X.dropna().unique()) - feature_range

set()

In [44]:
X.unique()

array(['a', nan, 'b'], dtype=object)

In [33]:
range_detector.fit(series=series_dict)
range_detector.series_values_range_

{'series_1': (np.float64(0.0001603758540575928),
  np.float64(0.9998402596145863)),
 'series_2': (np.float64(0.00022075129019427653),
  np.float64(0.9993174035672675))}

In [34]:
range_detector.fit(series=series_dict, exog=exog)
range_detector.series_values_range_
pprint(range_detector.exog_values_range_)

{'temp': (np.float64(0.00010689945941055878), np.float64(0.9999268883248396))}


# FIT

In [35]:
range_detector = RangeDriftDetector()
range_detector.fit(y)
pprint(range_detector.series_values_range_)
print(range_detector.series_names_in_)

{'y': (np.float64(0.00020584954214486473), np.float64(0.9993097405376111))}
['y']


In [36]:
range_detector = RangeDriftDetector()
range_detector.fit(series_df)
pprint(range_detector.series_values_range_)
print(range_detector.series_names_in_)

{'series_1': (np.float64(0.0001603758540575928),
              np.float64(0.9998402596145863)),
 'series_2': (np.float64(0.00022075129019427653),
              np.float64(0.9993174035672675))}
['series_1', 'series_2']


In [37]:
range_detector

RangeDriftDetector 
Series value ranges: {'series_1': (np.float64(0.0001603758540575928), np.float64(0.9998402596145863)), 'series_2': (np.float64(0.00022075129019427653), np.float64(0.9993174035672675))} 
Exogenous value ranges: {} 
Fitted series: ['series_1', 'series_2'] 
Fitted exogenous: [] 

In [38]:
range_detector = RangeDriftDetector()
range_detector.fit(series_dict)
pprint(range_detector.series_values_range_)
print(range_detector.series_names_in_)

{'series_1': (np.float64(0.0001603758540575928),
              np.float64(0.9998402596145863)),
 'series_2': (np.float64(0.00022075129019427653),
              np.float64(0.9993174035672675))}
['series_1', 'series_2']


In [39]:
range_detector = RangeDriftDetector()
range_detector.fit(series_df_long)
pprint(range_detector.series_values_range_)
print(range_detector.series_names_in_)

{'series_1': (np.float64(0.0001603758540575928),
              np.float64(0.9998402596145863)),
 'series_2': (np.float64(0.00022075129019427653),
              np.float64(0.9993174035672675))}
['series_1', 'series_2']


In [40]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog)
pprint(range_detector.exog_values_range_)
print(range_detector.exog_names_in_)

{'temp': (np.float64(0.00010689945941055878), np.float64(0.9999268883248396))}
['temp']


In [41]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog_df)
pprint(range_detector.exog_values_range_)
print(range_detector.exog_names_in_)

{'cat_feature_0': {'C', 'A', 'B'},
 'cat_feature_1': {'C', 'A', 'B'},
 'cat_feature_2': {'C', 'A', 'B'},
 'cat_feature_3': {'C', 'A', 'B'},
 'cat_feature_4': {'C', 'A', 'B'},
 'feature_0': (np.float64(0.0002416252688326459),
               np.float64(0.9998627048069476)),
 'feature_1': (np.float64(5.882069337292961e-05),
               np.float64(0.9998278346550146)),
 'feature_2': (np.float64(0.00023727698275222142),
               np.float64(0.9995271740708683)),
 'feature_3': (np.float64(8.066565786157831e-05),
               np.float64(0.999350944932692)),
 'feature_4': (np.float64(0.0008524529995808106),
               np.float64(0.9999230889409791))}
['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'cat_feature_0', 'cat_feature_1', 'cat_feature_2', 'cat_feature_3', 'cat_feature_4']


In [42]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog_dict)
pprint(range_detector.exog_values_range_)
print(range_detector.exog_names_in_)

{'series_1': {'cat_feature_0': {'C', 'A', 'B'},
              'cat_feature_1': {'C', 'A', 'B'},
              'cat_feature_2': {'C', 'A', 'B'},
              'cat_feature_3': {'C', 'A', 'B'},
              'cat_feature_4': {'C', 'A', 'B'},
              'feature_0': (np.float64(0.0002416252688326459),
                            np.float64(0.9998627048069476)),
              'feature_1': (np.float64(5.882069337292961e-05),
                            np.float64(0.9998278346550146)),
              'feature_2': (np.float64(0.00023727698275222142),
                            np.float64(0.9995271740708683)),
              'feature_3': (np.float64(8.066565786157831e-05),
                            np.float64(0.999350944932692)),
              'feature_4': (np.float64(0.0008524529995808106),
                            np.float64(0.9999230889409791))},
 'series_2': {'cat_feature_0': {'C', 'A', 'B'},
              'cat_feature_1': {'C', 'A', 'B'},
              'cat_feature_2': {'C', 'A', '

In [43]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog_df_long)
pprint(range_detector.exog_values_range_)
print(range_detector.exog_names_in_)

{'series_1': {'cat_feature_0': {'C', 'A', 'B'},
              'cat_feature_1': {'C', 'A', 'B'},
              'cat_feature_2': {'C', 'A', 'B'},
              'cat_feature_3': {'C', 'A', 'B'},
              'cat_feature_4': {'C', 'A', 'B'},
              'feature_0': (np.float64(0.0002416252688326459),
                            np.float64(0.9998627048069476)),
              'feature_1': (np.float64(5.882069337292961e-05),
                            np.float64(0.9998278346550146)),
              'feature_2': (np.float64(0.00023727698275222142),
                            np.float64(0.9995271740708683)),
              'feature_3': (np.float64(8.066565786157831e-05),
                            np.float64(0.999350944932692)),
              'feature_4': (np.float64(0.0008524529995808106),
                            np.float64(0.9999230889409791))},
 'series_2': {'cat_feature_0': {'C', 'A', 'B'},
              'cat_feature_1': {'C', 'A', 'B'},
              'cat_feature_2': {'C', 'A', '

# Predict

In [44]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog)
pprint(range_detector.series_values_range_)
pprint(range_detector.exog_values_range_)


{'y': (np.float64(0.00020584954214486473), np.float64(0.9993097405376111))}
{'temp': (np.float64(0.00010689945941055878), np.float64(0.9999268883248396))}


In [45]:
last_window_valid = y.tail(100).copy()
last_window_invalid = y.tail(100).copy()
last_window_invalid.iloc[5] = 10_000    
range_detector.predict(last_window=last_window_valid)
range_detector.predict(last_window=last_window_invalid)
range_detector


╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'y' has one or more values outside the range seen during training [0.00021,          │
│ 0.99931]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :176                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

RangeDriftDetector 
Series value ranges: {'y': (np.float64(0.00020584954214486473), np.float64(0.9993097405376111))} 
Exogenous value ranges: {'temp': (np.float64(0.00010689945941055878), np.float64(0.9999268883248396))} 
Fitted series: ['y'] 
Fitted exogenous: ['temp'] 

In [46]:
range_detector.fit(series_df)
last_window_valid_df = series_df.tail(100).copy()
last_window_invalid_df = series_df.tail(100).copy()
last_window_invalid_df.iloc[1, [0, 1]] = 10_000
range_detector.predict(last_window=last_window_valid_df)
range_detector.predict(last_window=last_window_invalid_df)
range_detector

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_1' has one or more values outside the range seen during training [0.00016,   │
│ 0.99984]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :176                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_2' has one or more values outside the range seen during training [0.00022,   │
│ 0.99932]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :176                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

RangeDriftDetector 
Series value ranges: {'series_1': (np.float64(0.0001603758540575928), np.float64(0.9998402596145863)), 'series_2': (np.float64(0.00022075129019427653), np.float64(0.9993174035672675))} 
Exogenous value ranges: {} 
Fitted series: ['series_1', 'series_2'] 
Fitted exogenous: [] 

In [47]:
range_detector.fit(series_dict)
last_window_valid_dict = {key: val.tail(100).copy() for key, val in series_dict.items()}
last_window_invalid_dict = {key: val.tail(100).copy() for key, val in series_dict.items()}
for key in last_window_invalid_dict.keys():
    last_window_invalid_dict[key].iloc[1] = 10_000
range_detector.predict(last_window=last_window_valid_dict)
range_detector.predict(last_window=last_window_invalid_dict)
range_detector

RangeDriftDetector 
Series value ranges: {'series_1': (np.float64(0.0001603758540575928), np.float64(0.9998402596145863)), 'series_2': (np.float64(0.00022075129019427653), np.float64(0.9993174035672675))} 
Exogenous value ranges: {} 
Fitted series: ['series_1', 'series_2'] 
Fitted exogenous: [] 

In [48]:
range_detector.fit(series_df_long)
last_window_valid_df_long= series_df_long.groupby(level=0).tail(100).copy()
last_window_invalid_df_long = series_df_long.groupby(level=0).tail(100).copy()
for key in last_window_invalid_df_long.index.get_level_values(0).unique():
    last_window_invalid_df_long.loc[key, pd.Timestamp("2013-06-02")] = 10_000
range_detector.predict(last_window=last_window_valid_df_long)
range_detector.predict(last_window=last_window_invalid_df_long)

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_1' has one or more values outside the range seen during training [0.00016,   │
│ 0.99984]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :176                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_2' has one or more values outside the range seen during training [0.00022,   │
│ 0.99932]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :176                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

In [49]:
range_detector.fit(series_df, exog=exog)
print(range_detector)
exog_valid = exog.tail(100).copy()
exog_invalid = exog.tail(100).copy()
exog_invalid.iloc[5] = 10_000
range_detector.predict(exog=exog_valid)
range_detector.predict(exog=exog_invalid)

RangeDriftDetector 
Series value ranges: {'series_1': (np.float64(0.0001603758540575928), np.float64(0.9998402596145863)), 'series_2': (np.float64(0.00022075129019427653), np.float64(0.9993174035672675))} 
Exogenous value ranges: {'temp': (np.float64(0.00010689945941055878), np.float64(0.9999268883248396))} 
Fitted series: ['series_1', 'series_2'] 
Fitted exogenous: ['temp'] 



╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'temp' has one or more values outside the range seen during training [0.00011,       │
│ 0.99993]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :176                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

In [50]:
range_detector = RangeDriftDetector()
range_detector.fit(series=y, exog=exog_df)
range_detector
exog_valid = exog_df.tail(100).copy()
exog_invalid = exog_df.tail(100).copy()
exog_invalid.iloc[5, [0,1]] = 10_000
range_detector.predict(exog=exog_valid)
range_detector.predict(exog=exog_invalid)

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'feature_0' has one or more values outside the range seen during training [0.00024,  │
│ 0.99986]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :176                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'feature_1' has one or more values outside the range seen during training [0.00006,  │
│ 0.99983]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :176                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

In [51]:
range_detector = RangeDriftDetector()
range_detector.fit(series=y, exog=exog_df)
range_detector
exog_valid = exog_df.tail(100).copy()
exog_invalid = exog_df.tail(100).copy()
exog_invalid.iloc[5, [0,1]] = 10_000
range_detector.predict(exog=exog_valid)
range_detector.predict(exog=exog_invalid)

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'feature_0' has one or more values outside the range seen during training [0.00024,  │
│ 0.99986]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :176                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'feature_1' has one or more values outside the range seen during training [0.00006,  │
│ 0.99983]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :176                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
range_detector = RangeDriftDetector()

range_detector.fit(series=series_dict, exog=exog_dict)
print(range_detector)
exog_valid = deepcopy(exog_dict)
exog_invalid = deepcopy(exog_dict)
for key in exog_invalid.keys():
    
    exog_invalid[key].iloc[1, [0,1]] = 10_000
range_detector.predict(exog=exog_valid)
range_detector.predict(exog=exog_invalid)
range_detector

RangeDriftDetector 
Series value ranges: {'series_1': (np.float64(0.0001603758540575928), np.float64(0.9998402596145863)), 'series_2': (np.float64(0.00022075129019427653), np.float64(0.9993174035672675))} 
Exogenous value ranges: {'series_1': {'feature_0': (np.float64(0.0002416252688326459), np.float64(0.9998627048069476)), 'feature_1': (np.float64(5.882069337292961e-05), np.float64(0.9998278346550146)), 'feature_2': (np.float64(0.00023727698275222142), np.float64(0.9995271740708683)), 'feature_3': (np.float64(8.066565786157831e-05), np.float64(0.999350944932692)), 'feature_4': (np.float64(0.0008524529995808106), np.float64(0.9999230889409791)), 'cat_feature_0': {'C', 'A', 'B'}, 'cat_feature_1': {'C', 'A', 'B'}, 'cat_feature_2': {'C', 'A', 'B'}, 'cat_feature_3': {'C', 'A', 'B'}, 'cat_feature_4': {'C', 'A', 'B'}}, 'series_2': {'feature_0': (np.float64(0.0002416252688326459), np.float64(0.9998627048069476)), 'feature_1': (np.float64(5.882069337292961e-05), np.float64(0.9998278346550146))

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'feature_1000' was not seen during training. Its range is unknown.                   │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location : <string>:46                                                               │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_1': 'feature_0' has one or more values outside the range seen during         │
│ training [0.00024, 0.99986]. This may affect the accuracy of the predictions.        │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :176                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_1': 'feature_1' has one or more values outside the range seen during         │
│ training [0.00006, 0.99983]. This may affect the accuracy of the predictions.        │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :176                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_2': 'feature_0' has one or more values outside the range seen during         │
│ training [0.00024, 0.99986]. This may affect the accuracy of the predictions.        │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :176                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_2': 'feature_1' has one or more values outside the range seen during         │
│ training [0.00006, 0.99983]. This may affect the accuracy of the predictions.        │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :176                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

RangeDriftDetector 
Series value ranges: {'series_1': (np.float64(0.0001603758540575928), np.float64(0.9998402596145863)), 'series_2': (np.float64(0.00022075129019427653), np.float64(0.9993174035672675))} 
Exogenous value ranges: {'series_1': {'feature_0': (np.float64(0.0002416252688326459), np.float64(0.9998627048069476)), 'feature_1': (np.float64(5.882069337292961e-05), np.float64(0.9998278346550146)), 'feature_2': (np.float64(0.00023727698275222142), np.float64(0.9995271740708683)), 'feature_3': (np.float64(8.066565786157831e-05), np.float64(0.999350944932692)), 'feature_4': (np.float64(0.0008524529995808106), np.float64(0.9999230889409791)), 'cat_feature_0': {'C', 'A', 'B'}, 'cat_feature_1': {'C', 'A', 'B'}, 'cat_feature_2': {'C', 'A', 'B'}, 'cat_feature_3': {'C', 'A', 'B'}, 'cat_feature_4': {'C', 'A', 'B'}}, 'series_2': {'feature_0': (np.float64(0.0002416252688326459), np.float64(0.9998627048069476)), 'feature_1': (np.float64(5.882069337292961e-05), np.float64(0.9998278346550146))

In [54]:
exog_valid['series_1']['feature_1000'] = 10000